In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error

In [2]:
train_joke_df = pd.read_csv(r'..\data\recsys-in-practice\train_joke_df.csv')
test_joke_df_nofactrating = pd.read_csv(r'..\data\recsys-in-practice\test_joke_df_nofactrating.csv', index_col=0)

In [3]:
train_joke_df["UID"] = train_joke_df["UID"].astype(int)
train_joke_df["JID"] = train_joke_df["JID"].astype(int)

In [4]:
train_joke_df["UID"] = train_joke_df["UID"] - 1
train_joke_df["JID"] = train_joke_df["JID"] - 1

In [5]:
npusers_arr = np.zeros((24983, 100), dtype=np.half)

for row in tqdm(train_joke_df.values):
    npusers_arr[int(row[0]), int(row[1])] = row[2]

  0%|          | 0/1448364 [00:00<?, ?it/s]

In [6]:
users_df = pd.DataFrame(npusers_arr, index=range(0, npusers_arr.shape[0]), columns=[f'j_{i}' for i in range(100)])
users_df

,j_0,j_1,j_2,j_3,j_4,j_5,j_6,j_7,j_8,j_9,...,j_90,j_91,j_92,j_93,j_94,j_95,j_96,j_97,j_98,j_99
0,-7.820312,8.789062,-9.656250,-8.156250,-7.519531,-8.500000,-9.851562,4.171875,-8.976562,-4.761719,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-5.628906,0.000000,0.000000,0.000000
1,4.078125,-0.290039,6.359375,4.371094,0.000000,-9.656250,-0.729980,-5.339844,8.882812,9.218750,...,2.820312,-4.949219,-0.290039,7.859375,-0.189941,0.000000,3.060547,0.000000,-4.320312,0.000000
2,0.000000,0.000000,0.000000,0.000000,9.031250,9.273438,9.031250,9.273438,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,8.351562,0.000000,0.000000,1.799805,0.000000,-2.820312,6.210938,0.000000,1.839844,...,0.000000,0.000000,0.000000,0.529785,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,8.500000,4.609375,-4.171875,-5.390625,0.000000,0.000000,7.039062,0.000000,-0.439941,0.000000,...,0.000000,5.578125,4.269531,5.191406,5.730469,1.549805,0.000000,6.550781,1.799805,1.599609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,0.439941,0.000000,0.000000,2.330078,0.000000,6.750000,-8.789062,-0.529785,0.000000,0.000000,...,8.828125,-1.209961,9.218750,-6.699219,0.000000,9.031250,6.550781,8.687500,0.000000,7.429688
24979,0.000000,-8.156250,8.593750,9.078125,0.870117,-8.929688,-3.500000,5.781250,-8.109375,0.000000,...,-1.169922,-5.730469,0.000000,0.239990,9.218750,-8.203125,0.000000,-8.593750,9.132812,8.453125
24980,0.000000,0.000000,0.000000,0.000000,-7.769531,0.000000,0.000000,-6.750000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
24981,0.000000,0.000000,0.000000,0.000000,-9.710938,0.000000,4.558594,-8.296875,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [7]:
train_joke_df = train_joke_df.merge(users_df, how='inner', left_on='UID',  right_index=True)
train_joke_df

,UID,JID,Rating,j_0,j_1,j_2,j_3,j_4,j_5,j_6,...,j_90,j_91,j_92,j_93,j_94,j_95,j_96,j_97,j_98,j_99
0,18028,5,-1.26,0.0,0.0,0.0,0.0,1.120117,-1.259766,3.300781,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
18990,18028,39,8.30,0.0,0.0,0.0,0.0,1.120117,-1.259766,3.300781,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
50486,18028,6,3.30,0.0,0.0,0.0,0.0,1.120117,-1.259766,3.300781,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
69643,18028,15,-9.27,0.0,0.0,0.0,0.0,1.120117,-1.259766,3.300781,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
124589,18028,14,8.11,0.0,0.0,0.0,0.0,1.120117,-1.259766,3.300781,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1303561,21378,61,0.15,0.0,0.0,0.0,0.0,-2.820312,0.000000,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0
1331118,21378,45,1.31,0.0,0.0,0.0,0.0,-2.820312,0.000000,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0
1368576,21378,48,-0.68,0.0,0.0,0.0,0.0,-2.820312,0.000000,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0
1376073,21378,53,2.33,0.0,0.0,0.0,0.0,-2.820312,0.000000,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0


In [8]:
for i, row in enumerate(tqdm(train_joke_df.values)):
    j = int(row[1])
    #print(train_joke_df.iloc[i, 3 + j])
    assert np.isclose(row[2], train_joke_df.iloc[i, 3 + j], atol=1e-2)
    train_joke_df.iloc[i, 3 + j] = 0
    #print(train_joke_df.iloc[i, 3 + j])


  0%|          | 0/1448364 [00:00<?, ?it/s]

In [9]:
for i, row in enumerate(tqdm(train_joke_df.values)):
    j = int(row[1])
    #print(train_joke_df.iloc[i, 3 + j])
    #print(row[3 + joke_id])
    assert np.isclose(0, train_joke_df.iloc[i, 3 + j])


  0%|          | 0/1448364 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
train_joke_df

In [ ]:
train_joke_df.tail(30)

In [10]:
rows, r_pos = np.unique(train_joke_df.values[:,0], return_inverse=True)
cols, c_pos = np.unique(train_joke_df.values[:,1], return_inverse=True)

In [11]:
rows

array([0.0000e+00, 1.0000e+00, 2.0000e+00, ..., 2.4980e+04, 2.4981e+04,
       2.4982e+04])

In [12]:
cols

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38.,
       39., 40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51.,
       52., 53., 54., 55., 56., 57., 58., 59., 60., 61., 62., 63., 64.,
       65., 66., 67., 68., 69., 70., 71., 72., 73., 74., 75., 76., 77.,
       78., 79., 80., 81., 82., 83., 84., 85., 86., 87., 88., 89., 90.,
       91., 92., 93., 94., 95., 96., 97., 98., 99.])

In [13]:
train_df, valid_df, train_queries, valid_queries = train_test_split(train_joke_df, r_pos, test_size=0.01, random_state=42)

In [14]:
len(valid_df)

14484

In [15]:
valid_df[valid_df['UID'] == 44]

,UID,JID,Rating,j_0,j_1,j_2,j_3,j_4,j_5,j_6,...,j_90,j_91,j_92,j_93,j_94,j_95,j_96,j_97,j_98,j_99


In [16]:
train_df[train_df['UID'] == 44]

,UID,JID,Rating,j_0,j_1,j_2,j_3,j_4,j_5,j_6,...,j_90,j_91,j_92,j_93,j_94,j_95,j_96,j_97,j_98,j_99
51504,44,71,3.20,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
693981,44,22,8.83,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
1239916,44,76,3.35,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
826257,44,6,4.51,8.789062,0.0,5.488281,0.919922,0.0,0.0,0.000000,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
1089514,44,78,2.77,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
650346,44,91,3.88,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,0.000000,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
1230384,44,30,5.39,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
746078,44,50,5.15,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
258762,44,86,2.48,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961


In [17]:
for i, row in enumerate(tqdm(valid_df.values)):
    u = int(row[0])
    j = int(row[1])
    
    train_df.loc[train_df['UID'] == u, f'j_{j}'] = 0
    valid_df.loc[valid_df['UID'] == u, f'j_{j}'] = 0

  0%|          | 0/14484 [00:00<?, ?it/s]

In [18]:
train_df[train_df['UID'] == 44]

,UID,JID,Rating,j_0,j_1,j_2,j_3,j_4,j_5,j_6,...,j_90,j_91,j_92,j_93,j_94,j_95,j_96,j_97,j_98,j_99
51504,44,71,3.20,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
693981,44,22,8.83,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
1239916,44,76,3.35,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
826257,44,6,4.51,8.789062,0.0,5.488281,0.919922,0.0,0.0,0.000000,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
1089514,44,78,2.77,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
650346,44,91,3.88,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,0.000000,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
1230384,44,30,5.39,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
746078,44,50,5.15,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
258762,44,86,2.48,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961


In [19]:
valid_df[valid_df['UID'] == 44]

,UID,JID,Rating,j_0,j_1,j_2,j_3,j_4,j_5,j_6,...,j_90,j_91,j_92,j_93,j_94,j_95,j_96,j_97,j_98,j_99


In [20]:
from catboost import CatBoostRanker, Pool, MetricVisualizer, CatBoostRegressor
from copy import deepcopy

In [21]:
train_df.drop(columns=['Rating'])

,UID,JID,j_0,j_1,j_2,j_3,j_4,j_5,j_6,j_7,...,j_90,j_91,j_92,j_93,j_94,j_95,j_96,j_97,j_98,j_99
1228743,585,14,0.000000,0.729980,-2.859375,-0.340088,0.000000,-2.089844,-0.680176,-0.970215,...,2.619141,0.000000,0.000000,0.000000,0.000000,0.000000,2.570312,0.000000,0.000000,0.000000
847495,348,65,3.500000,-4.851562,5.828125,-4.171875,0.000000,3.300781,-6.308594,5.730469,...,0.000000,0.000000,1.309570,0.000000,0.000000,-3.980469,0.000000,0.000000,0.000000,0.000000
1165753,6924,69,-9.851562,-9.367188,6.261719,1.360352,-2.179688,-9.132812,-8.882812,0.870117,...,0.000000,-6.988281,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-8.296875,0.000000
358457,16488,62,4.949219,0.000000,9.218750,0.000000,-0.049988,0.000000,0.000000,9.031250,...,0.000000,8.929688,0.000000,-3.830078,0.000000,0.000000,0.000000,7.960938,4.320312,9.031250
766031,10155,35,6.988281,5.000000,0.870117,-0.629883,0.000000,8.296875,4.218750,0.000000,...,0.000000,7.769531,0.000000,0.000000,0.000000,1.019531,3.589844,1.410156,3.640625,2.039062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193841,24455,41,0.000000,2.230469,-0.290039,1.650391,0.919922,3.500000,0.000000,1.209961,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
768014,864,20,0.000000,0.290039,0.000000,0.000000,3.199219,0.000000,4.761719,4.171875,...,0.000000,0.000000,0.000000,0.000000,1.839844,0.000000,0.000000,0.000000,0.000000,0.000000
661501,21881,62,0.000000,0.000000,-0.290039,7.281250,2.039062,8.828125,-0.870117,-2.039062,...,6.500000,5.730469,8.828125,0.000000,8.828125,2.619141,7.281250,8.250000,0.000000,-0.290039
1192804,24548,4,0.000000,0.000000,0.000000,0.000000,0.000000,3.109375,7.140625,2.330078,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [22]:
cat_features = ['UID', 'JID']

In [23]:
train_pool = Pool(train_df.drop(columns=['Rating']), label=train_df['Rating'], cat_features=cat_features)
valid_pool = Pool(valid_df.drop(columns=['Rating']), label=valid_df['Rating'], cat_features=cat_features)
main_pool = Pool(train_joke_df.drop(columns=['Rating']), label=train_joke_df['Rating'], cat_features=cat_features)

test_pool = Pool(test_joke_df_nofactrating, cat_features=cat_features)

In [24]:
#train = Pool(
#    data=X_train,
#    label=y_train,
#    group_id=queries_train,
#    cat_features=[0, 1]
#)

#test = Pool(
#    data=X_test,
#    label=y_test,
#    group_id=queries_test,
#    cat_features=[0, 1]
#)

In [25]:
default_parameters = {
    'iterations': 1000,
    'custom_metric': 'RMSE',
    'random_seed': 0,
    'train_dir':'RMSE',
    'objective':'RMSE',
    'loss_function':'RMSE',
    'eval_metric':'RMSE',
}


In [26]:
model = CatBoostRegressor(**default_parameters)
model.fit(train_pool, eval_set=valid_pool, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.159393
0:	learn: 5.0563513	test: 5.0688616	best: 5.0688616 (0)	total: 497ms	remaining: 8m 16s
1:	learn: 4.9227374	test: 4.9266738	best: 4.9266738 (1)	total: 918ms	remaining: 7m 38s
2:	learn: 4.8209325	test: 4.8177296	best: 4.8177296 (2)	total: 1.17s	remaining: 6m 29s
3:	learn: 4.7423051	test: 4.7338392	best: 4.7338392 (3)	total: 1.71s	remaining: 7m 4s
4:	learn: 4.6826277	test: 4.6713877	best: 4.6713877 (4)	total: 1.95s	remaining: 6m 27s
5:	learn: 4.6334501	test: 4.6208814	best: 4.6208814 (5)	total: 2.33s	remaining: 6m 26s
6:	learn: 4.5925072	test: 4.5864148	best: 4.5864148 (6)	total: 2.75s	remaining: 6m 29s
7:	learn: 4.5587702	test: 4.5571399	best: 4.5571399 (7)	total: 3.15s	remaining: 6m 30s
8:	learn: 4.5294917	test: 4.5339926	best: 4.5339926 (8)	total: 3.54s	remaining: 6m 29s
9:	learn: 4.5043574	test: 4.5138090	best: 4.5138090 (9)	total: 3.95s	remaining: 6m 30s
10:	learn: 4.4808201	test: 4.4971803	best: 4.4971803 (10)	total: 4.33s	remaining: 6m 29s
11:	learn: 4

93:	learn: 3.9311346	test: 4.3346186	best: 4.3333865 (91)	total: 45.3s	remaining: 7m 16s
94:	learn: 3.9289884	test: 4.3338180	best: 4.3333865 (91)	total: 45.7s	remaining: 7m 15s
95:	learn: 3.9227454	test: 4.3331863	best: 4.3331863 (95)	total: 46.2s	remaining: 7m 15s
96:	learn: 3.9187470	test: 4.3319168	best: 4.3319168 (96)	total: 46.6s	remaining: 7m 13s
97:	learn: 3.9132793	test: 4.3314107	best: 4.3314107 (97)	total: 47s	remaining: 7m 12s
98:	learn: 3.9076531	test: 4.3314258	best: 4.3314107 (97)	total: 47.5s	remaining: 7m 12s
99:	learn: 3.9046545	test: 4.3312679	best: 4.3312679 (99)	total: 48s	remaining: 7m 11s
100:	learn: 3.8985457	test: 4.3314234	best: 4.3312679 (99)	total: 48.4s	remaining: 7m 10s
101:	learn: 3.8959046	test: 4.3312117	best: 4.3312117 (101)	total: 48.9s	remaining: 7m 10s
102:	learn: 3.8900799	test: 4.3300242	best: 4.3300242 (102)	total: 49.6s	remaining: 7m 11s
103:	learn: 3.8866946	test: 4.3287981	best: 4.3287981 (103)	total: 50.1s	remaining: 7m 11s
104:	learn: 3.8809

184:	learn: 3.5763380	test: 4.3111032	best: 4.3111032 (184)	total: 1m 26s	remaining: 6m 18s
185:	learn: 3.5727805	test: 4.3107374	best: 4.3107374 (185)	total: 1m 26s	remaining: 6m 18s
186:	learn: 3.5703094	test: 4.3106016	best: 4.3106016 (186)	total: 1m 27s	remaining: 6m 18s
187:	learn: 3.5690174	test: 4.3100580	best: 4.3100580 (187)	total: 1m 27s	remaining: 6m 17s
188:	learn: 3.5630909	test: 4.3106767	best: 4.3100580 (187)	total: 1m 27s	remaining: 6m 16s
189:	learn: 3.5593326	test: 4.3109810	best: 4.3100580 (187)	total: 1m 28s	remaining: 6m 15s
190:	learn: 3.5570907	test: 4.3107411	best: 4.3100580 (187)	total: 1m 28s	remaining: 6m 15s
191:	learn: 3.5516911	test: 4.3100552	best: 4.3100552 (191)	total: 1m 29s	remaining: 6m 15s
192:	learn: 3.5495179	test: 4.3092315	best: 4.3092315 (192)	total: 1m 29s	remaining: 6m 14s
193:	learn: 3.5468068	test: 4.3089165	best: 4.3089165 (193)	total: 1m 29s	remaining: 6m 13s
194:	learn: 3.5424577	test: 4.3085205	best: 4.3085205 (194)	total: 1m 30s	remain

274:	learn: 3.3407564	test: 4.3060177	best: 4.3048202 (255)	total: 2m 6s	remaining: 5m 32s
275:	learn: 3.3399544	test: 4.3059965	best: 4.3048202 (255)	total: 2m 6s	remaining: 5m 31s
276:	learn: 3.3381139	test: 4.3063745	best: 4.3048202 (255)	total: 2m 6s	remaining: 5m 31s
277:	learn: 3.3358579	test: 4.3063646	best: 4.3048202 (255)	total: 2m 7s	remaining: 5m 30s
278:	learn: 3.3345948	test: 4.3061541	best: 4.3048202 (255)	total: 2m 7s	remaining: 5m 30s
279:	learn: 3.3330091	test: 4.3060734	best: 4.3048202 (255)	total: 2m 8s	remaining: 5m 29s
280:	learn: 3.3304286	test: 4.3063015	best: 4.3048202 (255)	total: 2m 8s	remaining: 5m 29s
281:	learn: 3.3283502	test: 4.3061918	best: 4.3048202 (255)	total: 2m 9s	remaining: 5m 28s
282:	learn: 3.3268545	test: 4.3063789	best: 4.3048202 (255)	total: 2m 9s	remaining: 5m 27s
283:	learn: 3.3227106	test: 4.3064007	best: 4.3048202 (255)	total: 2m 9s	remaining: 5m 27s
284:	learn: 3.3204579	test: 4.3061649	best: 4.3048202 (255)	total: 2m 10s	remaining: 5m 26

...

634:	learn: 2.7102586	test: 4.3549179	best: 4.3048202 (255)	total: 4m 56s	remaining: 2m 50s
635:	learn: 2.7084774	test: 4.3551538	best: 4.3048202 (255)	total: 4m 56s	remaining: 2m 49s
636:	learn: 2.7074391	test: 4.3555653	best: 4.3048202 (255)	total: 4m 57s	remaining: 2m 49s
637:	learn: 2.7071854	test: 4.3558144	best: 4.3048202 (255)	total: 4m 57s	remaining: 2m 48s
638:	learn: 2.7033499	test: 4.3559712	best: 4.3048202 (255)	total: 4m 58s	remaining: 2m 48s
639:	learn: 2.7020633	test: 4.3562140	best: 4.3048202 (255)	total: 4m 58s	remaining: 2m 48s
640:	learn: 2.7014640	test: 4.3564757	best: 4.3048202 (255)	total: 4m 59s	remaining: 2m 47s
641:	learn: 2.7003645	test: 4.3567875	best: 4.3048202 (255)	total: 4m 59s	remaining: 2m 47s
642:	learn: 2.6996409	test: 4.3569739	best: 4.3048202 (255)	total: 4m 59s	remaining: 2m 46s
643:	learn: 2.6973954	test: 4.3570827	best: 4.3048202 (255)	total: 5m	remaining: 2m 46s
644:	learn: 2.6961811	test: 4.3573347	best: 4.3048202 (255)	total: 5m	remaining: 2m 

724:	learn: 2.5855906	test: 4.3703892	best: 4.3048202 (255)	total: 5m 40s	remaining: 2m 9s
725:	learn: 2.5847023	test: 4.3707314	best: 4.3048202 (255)	total: 5m 40s	remaining: 2m 8s
726:	learn: 2.5839751	test: 4.3709363	best: 4.3048202 (255)	total: 5m 41s	remaining: 2m 8s
727:	learn: 2.5826653	test: 4.3712138	best: 4.3048202 (255)	total: 5m 42s	remaining: 2m 7s
728:	learn: 2.5810691	test: 4.3712226	best: 4.3048202 (255)	total: 5m 42s	remaining: 2m 7s
729:	learn: 2.5807314	test: 4.3712977	best: 4.3048202 (255)	total: 5m 43s	remaining: 2m 6s
730:	learn: 2.5793508	test: 4.3718946	best: 4.3048202 (255)	total: 5m 43s	remaining: 2m 6s
731:	learn: 2.5783657	test: 4.3722735	best: 4.3048202 (255)	total: 5m 44s	remaining: 2m 6s
732:	learn: 2.5774539	test: 4.3723366	best: 4.3048202 (255)	total: 5m 44s	remaining: 2m 5s
733:	learn: 2.5772710	test: 4.3726753	best: 4.3048202 (255)	total: 5m 45s	remaining: 2m 5s
734:	learn: 2.5748666	test: 4.3727227	best: 4.3048202 (255)	total: 5m 45s	remaining: 2m 4s

814:	learn: 2.4388331	test: 4.3919595	best: 4.3048202 (255)	total: 6m 25s	remaining: 1m 27s
815:	learn: 2.4381753	test: 4.3923599	best: 4.3048202 (255)	total: 6m 25s	remaining: 1m 27s
816:	learn: 2.4376470	test: 4.3926491	best: 4.3048202 (255)	total: 6m 26s	remaining: 1m 26s
817:	learn: 2.4343963	test: 4.3927018	best: 4.3048202 (255)	total: 6m 26s	remaining: 1m 26s
818:	learn: 2.4320750	test: 4.3928945	best: 4.3048202 (255)	total: 6m 27s	remaining: 1m 25s
819:	learn: 2.4315255	test: 4.3931583	best: 4.3048202 (255)	total: 6m 27s	remaining: 1m 25s
820:	learn: 2.4294870	test: 4.3932976	best: 4.3048202 (255)	total: 6m 28s	remaining: 1m 24s
821:	learn: 2.4288145	test: 4.3934812	best: 4.3048202 (255)	total: 6m 28s	remaining: 1m 24s
822:	learn: 2.4273960	test: 4.3937053	best: 4.3048202 (255)	total: 6m 29s	remaining: 1m 23s
823:	learn: 2.4258289	test: 4.3938807	best: 4.3048202 (255)	total: 6m 30s	remaining: 1m 23s
824:	learn: 2.4213727	test: 4.3941792	best: 4.3048202 (255)	total: 6m 30s	remain

905:	learn: 2.3210976	test: 4.4143356	best: 4.3048202 (255)	total: 7m 10s	remaining: 44.7s
906:	learn: 2.3206162	test: 4.4142589	best: 4.3048202 (255)	total: 7m 11s	remaining: 44.2s
907:	learn: 2.3190795	test: 4.4145385	best: 4.3048202 (255)	total: 7m 11s	remaining: 43.8s
908:	learn: 2.3181549	test: 4.4147005	best: 4.3048202 (255)	total: 7m 12s	remaining: 43.3s
909:	learn: 2.3175987	test: 4.4149206	best: 4.3048202 (255)	total: 7m 12s	remaining: 42.8s
910:	learn: 2.3163328	test: 4.4152178	best: 4.3048202 (255)	total: 7m 13s	remaining: 42.3s
911:	learn: 2.3150095	test: 4.4154268	best: 4.3048202 (255)	total: 7m 14s	remaining: 41.9s
912:	learn: 2.3127687	test: 4.4154325	best: 4.3048202 (255)	total: 7m 14s	remaining: 41.4s
913:	learn: 2.3124005	test: 4.4157486	best: 4.3048202 (255)	total: 7m 15s	remaining: 40.9s
914:	learn: 2.3119982	test: 4.4160683	best: 4.3048202 (255)	total: 7m 15s	remaining: 40.5s
915:	learn: 2.3112313	test: 4.4163696	best: 4.3048202 (255)	total: 7m 16s	remaining: 40s
9

996:	learn: 2.2289821	test: 4.4346651	best: 4.3048202 (255)	total: 7m 56s	remaining: 1.43s
997:	learn: 2.2285876	test: 4.4349675	best: 4.3048202 (255)	total: 7m 57s	remaining: 957ms
998:	learn: 2.2276868	test: 4.4351960	best: 4.3048202 (255)	total: 7m 57s	remaining: 478ms
999:	learn: 2.2266906	test: 4.4353780	best: 4.3048202 (255)	total: 7m 58s	remaining: 0us

bestTest = 4.304820233
bestIteration = 255

Shrink model to first 256 iterations.


In [27]:
assert False

AssertionError: 

In [28]:
predict = model.predict(valid_pool)
print(mean_squared_error(valid_df['Rating'].values, predict, squared=False))


4.304820226440547


In [ ]:
predict = model.predict(test_pool)

test_joke_df_nofactrating['Rating'] = predict

display(test_joke_df_nofactrating['Rating'].to_frame().head(5))
test_joke_df_nofactrating['Rating'].to_frame().to_csv('catboost0.9.csv')

In [32]:
test_joke_df_nofactrating = pd.read_csv(r'..\data\recsys-in-practice\test_joke_df_nofactrating.csv', index_col=0)

test_joke_df_nofactrating = test_joke_df_nofactrating.merge(users_df, how='left', left_on='UID',  right_index=True)



test_pool = Pool(test_joke_df_nofactrating, cat_features=cat_features)

predict = model.predict(test_pool)

test_joke_df_nofactrating['Rating'] = predict

display(test_joke_df_nofactrating['Rating'].to_frame().head(5))
test_joke_df_nofactrating['Rating'].to_frame().to_csv('catboost_with_rating.csv')

,Rating
InteractionID,
0,3.129127
1,0.014695
2,-3.035039
3,1.548267
4,-0.077219


In [33]:
test_joke_df_nofactrating

,UID,JID,j_0,j_1,j_2,j_3,j_4,j_5,j_6,j_7,...,j_91,j_92,j_93,j_94,j_95,j_96,j_97,j_98,j_99,Rating
InteractionID,,,,,,,,,,,,,,,,,,,,,
0,11228,39,0.000000,0.000000,0.000000,0.000000,-9.812500,8.500000,-9.218750,-0.779785,...,9.171875,5.531250,0.000000,0.000000,0.000000,9.218750,0.000000,0.000000,0.000000,3.129127
1,21724,85,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-9.562500,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014695
2,16782,56,-0.099976,0.049988,-4.371094,0.000000,-0.680176,3.349609,-8.250000,-2.910156,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.035039
3,12105,42,4.851562,2.380859,1.889648,-1.940430,0.000000,4.128906,4.320312,-6.359375,...,-4.558594,5.148438,3.060547,-4.078125,4.660156,3.689453,0.489990,5.101562,0.000000,1.548267
4,14427,2,0.830078,9.367188,-9.507812,-9.656250,-4.421875,6.839844,-9.953125,7.570312,...,3.980469,0.000000,-9.851562,8.687500,-0.529785,-0.439941,-5.339844,0.000000,3.109375,-0.077219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362086,3085,66,3.300781,-6.800781,0.000000,0.000000,-7.140625,0.000000,-0.970215,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.049988,0.000000,-0.008576
362087,13765,31,3.160156,6.019531,5.441406,-0.580078,0.000000,0.000000,-3.109375,5.578125,...,0.000000,0.729980,-0.389893,1.019531,-1.549805,-3.880859,0.000000,1.839844,-0.099976,2.879349
362088,10341,29,0.000000,1.799805,-2.089844,-1.599609,0.729980,4.710938,0.150024,0.000000,...,1.650391,0.000000,1.500000,0.000000,1.650391,3.199219,5.050781,2.859375,-2.330078,0.945142


In [ ]:
parameters = {
    'iterations': 2000,
    'custom_metric': 'RMSE',
    'random_seed': 0,
    'train_dir':'RMSE',
    'objective':'RMSE',
    'loss_function':'RMSE',
    'eval_metric':'RMSE',
}

model = CatBoostRanker(**parameters)
model.fit(main_pool, eval_set=valid_pool, plot=True)

In [ ]:
predict = model.predict(valid_pool)
print(mean_squared_error(valid_df['Rating'].values, predict, squared=False))


In [ ]:
predict = model.predict(test_pool)

test_joke_df_nofactrating['Rating'] = predict

display(test_joke_df_nofactrating['Rating'].to_frame().head(5))
test_joke_df_nofactrating['Rating'].to_frame().to_csv('catboost.csv')